In [63]:
import pandas as pd

data = pd.read_csv('../public/cpi.csv')

data.columns = ['dummy', '연도'] + [str(year) for year in range(1970, 2024)]

data = data.set_index('연도').transpose() # 행 열을 돌려 반전시킨다

data = data.drop('dummy', axis=0)

# for i in range(2, len(data.columns)):
#     del data[i]

# data = data.drop('단위:', axis=0)
# data = data.drop('2020=100, %', axis=0)
# data = data.drop('2023', axis=0)

# data = data.rename_axis('연도', inplace=True)

data

연도,4.7,16.0,-,-,-,-,-,-,-,-,-,-,-,-,"통계청, 「소비자물가조사」","* 자료 : 통계청, 「소비자물가조사」 각 연도",1) 소비자물가상승률 = {(당해년도 연간소비자물가지수 ÷ 직전 연간소비자물가지수) - 1} × 100.
1970,5.3,13.5,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1971,5.9,11.7,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1972,6.1,3.2,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1973,7.6,24.3,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1974,9.5,25.2,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1975,10.9,15.3,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1976,12.0,10.1,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1977,13.8,14.5,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1978,16.3,18.3,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN
1979,21.0,28.7,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN


In [44]:
print(data)

None
